In [2]:
import sys
from pathlib import Path
import os

# Configuración crítica de paths - EJECUTAR PRIMERO
current_dir = Path.cwd()
print(f"Directorio actual: {current_dir}")

# Determina automáticamente la raíz del proyecto
if 'notebooks' in str(current_dir):
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir

SRC_DIR = PROJECT_ROOT / 'src'

# Verifica que existe src/models/
if not (SRC_DIR / 'models').exists():
    raise FileNotFoundError(f"No se encontró la carpeta src/models en {SRC_DIR}")

# Añade src al path de Python
sys.path.insert(0, str(SRC_DIR))
print(f"\nPaths de búsqueda actualizados:")
for p in sys.path[:3]:  # Muestra solo los primeros 3 paths
    print(f"- {p}")


from src.models.diagnosis import Diagnosis
from src.models.patient import Patient
from src.models.eye_examination import EyeExamination
from src.models.eye_image import EyeImage
from src.models.retina import Retina
from src.models.contours import Contours
import pandas as pd
import os

def convert_xls_to_df(path_file: pd.ExcelFile, sheet: str) -> pd.DataFrame:
    df = pd.read_excel(path_file, sheet_name=sheet)
    df_optimazing = optimazing_df(df)
    print(df_optimazing)
    return df_optimazing

def optimazing_df(df: pd.DataFrame) -> pd.DataFrame:
    df['ID'] = df['ID'].str.replace('#', '').astype('int')
    df['Age'] = df['Age'].astype('Int8')
    df['Gender'] = df['Gender'].astype('category')
    df['Diagnosis'] = df['Diagnosis'].astype('category')
    df['dioptre_1'] = df['dioptre_1'].astype('Float32')
    df['dioptre_2'] = df['dioptre_2'].astype('Float32')
    df['astigmatism'] = df['astigmatism'].astype('Float32')
    df['Phakic/Pseudophakic'] = df['Phakic/Pseudophakic'].astype('category')
    df['Pneumatic'] = df['Pneumatic'].astype('Float32')
    df['Perkins'] = df['Perkins'].astype('Float32')
    df['Pachymetry'] = df['Pachymetry'].astype('Float32')
    df['Axial_Length'] = df['Axial_Length'].astype('Float32')
    df['VF_MD'] = df['VF_MD'].astype('Float32')
    return df

def load_data(df_entry_od: str) -> list:
    try:
        diagnosises_od = []
        for index, row in df_entry_od.iterrows():
            diagnosis = Diagnosis(
                diagnosis_id=row['ID'],
                diagnosis=row['Diagnosis'],
            )
            patient = Patient(
                age=row['Age'],
                gender=row['Gender']
            )
            diagnosis.add_patient(patient=patient)
            eye_examination = EyeExamination(
                dioptre_1=row['dioptre_1'],
                dioptre_2=row['dioptre_2'],
                astigmatism=row['astigmatism'],
                phakic_pseudophakic=row['Phakic/Pseudophakic'],
                pneumatic=row['Pneumatic'],
                perkins=row['Perkins'],
                pachymetry=row['Pachymetry'],
                axial_length=row['Axial_Length'],
                vf_md=row['VF_MD']
            )
            diagnosis.add_eye_examination_od(eye_examination_od=eye_examination)
            eye_image = EyeImage(
                path_retina_contours_image=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'ImagesWithContours', f'Opht_cont_RET{row['ID']:03d}OD.jpg'),            
            )
            retina_image = Retina(
                path_retina_image=os.path.join('..', 'data', 'load_data', 'FundusImages', f'RET{row['ID']:03d}OD.jpg')
            )
            contours_layers = Contours(
                path_retina_cup_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_cup_exp1.txt'),
                path_retina_cup_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_cup_exp2.txt'),
                path_retina_disc_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_disc_exp1.txt'),
                path_retina_disc_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_disc_exp2.txt')
            )
            eye_examination.add_eye_image(eye_image=eye_image)
            eye_image.add_retina_image(retina_image=retina_image)
            eye_image.add_contours_layer(contours_layer=contours_layers)
            diagnosises_od.append(diagnosis)
    except Exception as e:
        print(e)
    else: 
        return diagnosises_od
    
def add_diagnosises_os(diagnosises_loaded: list[Diagnosis], df_entry_os: pd.DataFrame):
    for index, row_os in df_entry_os.iterrows():
        for diagnosis in diagnosises_loaded:
            if row_os['ID']==diagnosis.diagnosis_id:
                eye_examination = EyeExamination(
                    dioptre_1=row_os['dioptre_1'],
                    dioptre_2=row_os['dioptre_2'],
                    astigmatism=row_os['astigmatism'],
                    phakic_pseudophakic=row_os['Phakic/Pseudophakic'],
                    pneumatic=row_os['Pneumatic'],
                    perkins=row_os['Perkins'],
                    pachymetry=row_os['Pachymetry'],
                    axial_length=row_os['Axial_Length'],
                    vf_md=row_os['VF_MD']
                )
                diagnosis.add_eye_examination_os(eye_examination_os=eye_examination)
                eye_image = EyeImage(
                    path_retina_contours_image=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'ImagesWithContours', f'Opht_cont_RET{row_os['ID']:03d}OS.jpg'),            
                )
                retina_image = Retina(
                    path_retina_image=os.path.join('..', 'data', 'load_data', 'FundusImages', f'RET{row_os['ID']:03d}OS.jpg')
                )
                contours_layers = Contours(
                    path_retina_cup_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_cup_exp1.txt'),
                    path_retina_cup_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_cup_exp2.txt'),
                    path_retina_disc_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_disc_exp1.txt'),
                    path_retina_disc_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_disc_exp2.txt')
                )
                eye_examination.add_eye_image(eye_image=eye_image)
                eye_image.add_retina_image(retina_image=retina_image)
                eye_image.add_contours_layer(contours_layer=contours_layers)

Directorio actual: /Users/stevenvalverde/Documents/master ia/mod01-fundamentos/project/modulo1_proyecto_final/notebooks

Paths de búsqueda actualizados:
- /Users/stevenvalverde/Documents/master ia/mod01-fundamentos/project/modulo1_proyecto_final/src
- /opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip
- /opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13


In [3]:
from src.utils import convert_xls_to_df, load_data, add_diagnosises_os
import os

df = convert_xls_to_df(os.path.join('..', 'data', 'load_data', 'ClinicalData', 'diagnosis_data.xlsx'), 'od')
diagnosises_loaded = load_data(df)
df_os = convert_xls_to_df(os.path.join('..', 'data', 'load_data', 'ClinicalData', 'diagnosis_data.xlsx'), 'os')
add_diagnosises_os(diagnosises_loaded=diagnosises_loaded, df_entry_os=df_os)

      ID  Age Gender Diagnosis  dioptre_1  dioptre_2  astigmatism  \
0      2   47      0         2       0.75      -1.75         90.0   
1      4   58      1         1        1.5      -1.75         85.0   
2      5   89      1         1      -0.75      -1.25        101.0   
3      6   69      0         2        1.0       -1.5         95.0   
4      7   22      1         2      -0.25        0.0          0.0   
..   ...  ...    ...       ...        ...        ...          ...   
239  289   64      0         0        0.5       -1.0        120.0   
240  290   75      1         0       0.25      -0.25          5.0   
241  291   55      0         0       1.25      -1.25         92.0   
242  292   56      1         0       1.75       -1.5         73.0   
243  293   39      1         0      -0.25      -0.75        155.0   

    Phakic/Pseudophakic  Pneumatic  Perkins  Pachymetry  Axial_Length  VF_MD  
0                   0.0       21.0     <NA>       586.0     23.639999  -0.07  
1            